In [2]:
import os
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import RobustScaler

import cvxpy

def ensure_dir_exists(path):
    dirpath = os.path.dirname(path)
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

## Output in Weka format

In [41]:
import os
def predict_for_test(test, predict, probability, path):
    
    ensure_dir_exists(path)
    
    with open(path, 'w') as f:
        f.write("=== Predictions on test data ===\n")
        f.write(" inst#     actual  predicted error prediction\n")
        for i in range(len(test)):
            string = [str(i + 1)]
            if test[i] == 1:
                string.append("1:positive")
            else:
                string.append("2:negative")
            if predict[i] == 1:
                string.append("1:positive")
            else:
                string.append("2:negative")
            if test[i] == predict[i]:
                string.append(" " * 5)
            else:
                string.append(" " * 2 + "+" + " " * 2)
            if predict[i] == 1:
                string.append(str(probability[i][1]))
            else:
                string.append(str(probability[i][0]))
            string = " ".join(string) + "\n"
            f.write(string)   
    

## Load data

In [4]:
import arff

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

#import data
def importData(path):
#     dataset = arff.load(open(path, 'rb'))
    dataset = arff.load(open(path))    
    data = np.array(dataset['data'])
    #print data[:10]

    #extract features and labels
    features = []
    labels = []
    for d in data:
        f = []
        for i in range(len(d) - 1):
            num = float(d[i])
            if int(num) == num:
                num = int(num)
            f.append(num)
        features.append(f)

        if d[-1] == "positive":
            labels.append(1)
        else:
            labels.append(0)
    return np.asarray(features), np.asarray(labels)

---
## Word embedding vectors

In [5]:
# Map each word to an index
ndim = 300
glove_path = "../data/glove_embeddings/glove.6B.{}d.txt".format(ndim)
with open(glove_path, "rb") as lines:
    w2idx = {line.split()[0].decode("utf-8"): i for i, line in enumerate(lines)}

In [7]:
vectors = np.empty((len(w2idx), ndim), dtype=np.float)
with open(glove_path, "rb") as lines:
    for i, line in enumerate(lines):
        # Fix for Python 3
        vectors[i] = np.asarray(list(map(float, line.split()[1:])))

In [8]:
import string
from nltk.corpus import stopwords

words_to_exclude = frozenset(string.punctuation) | frozenset(["..", "..."])
words_to_exclude |= frozenset(stopwords.words("english"))

---

## Rank propagation

In [10]:
def get_QA_group_count(infile):        
    line = infile.readline().strip()
    if line == "":
        return None
    
    if not line.startswith("<QApairs"):
        raise Exception("Invalid data format: {}<-----".format(line))
    
    sentence_count = 0
    while not line.strip().startswith("</QApairs"):
        line = infile.readline().replace('\t', ' ')                    
        if line.strip().lower().startswith("<positive") or line.strip().lower().startswith("<negative"):
            sentence_count += 1
    
    return sentence_count

def get_QA_group_indicators(filepath):
    """Get vector indicating which example belongs to which question group
    """
    with open(filepath) as infile:
        indicators = []
        qn_number = 0
        while infile:
            count = get_QA_group_count(infile)

            # Check for EOF
            if count == None:
                break

            if count > 0:
                indicators += ([qn_number] * count)
                qn_number += 1
        
    return np.asarray(indicators)

# Get a question and its candidate answers
def get_QA_group(infile):
    question = []
    answers = []
    line = infile.readline().strip()
    if line == "":
        return None
    
    if not line.startswith("<QApairs"):
        raise Exception("Invalid data format: {}<-----".format(line))
        
    while not line.strip().startswith("</QApairs"):
        line = infile.readline().replace('\t', ' ')
        if line.strip().lower().startswith("<question"):
            line = infile.readline().replace('\t', ' ')
            question.append(line.strip())
        elif line.strip().lower().startswith("<positive"):
            line = infile.readline().replace('\t', ' ')
            answers.append(("positive", line.strip()))
        elif line.strip().lower().startswith("<negative"):
            line = infile.readline().replace('\t', ' ')
            answers.append(("negative", line.strip()))
    
    return {"question": question, "answers": answers}                

from nltk.tokenize import WhitespaceTokenizer

def extract_vector(sentence, exclude, w2idx, wordvectors):
    """Compute the vector for a sentence by averaging the words in the sentence that has word embeddings"""
    # Tokenize sentence
    splitter = WhitespaceTokenizer()
    tokens = splitter.tokenize(sentence)    
    # Remove stopwords and punctuation
    words = [t.lower() for t in tokens if t.lower() not in exclude ]
    
    # If we cannot find any words, we can consider returning a vector of 0
    # and set the resulting cosine similarity to 0 otherwise will result in nan
    # because cosine similarity will divide by 0.
    assert(len(words) > 0)
            
    # Average words in sentence that are in word matrix
    try:
        avg_vec = np.mean([wordvectors[w2idx[w]] for w in words if w in w2idx ] 
                                                 or [np.zeros(wordvectors.shape[1])], 
                           axis=0)
        if not np.any(avg_vec):
            print("Tokens cannot be found: {}".format(words))
        assert(np.any(avg_vec))
        return avg_vec
    except UnicodeDecodeError:
        print(line.strip())
        raise

def compute_pairwise_distance_matrix(X, k, p=2):
    """Compute pairwise distances between each point in X
    and its k-nearest neighbors."""

    from scipy.spatial import KDTree
    kdtree = KDTree(X)
    A = np.zeros((X.shape[0], X.shape[0]), dtype=np.float)
    for i, x in enumerate(X):
        distances, idxs = kdtree.query(x, k+1, p)  # k+1 as one pt is the pt itself.
        for d, j in zip(distances, idxs):
            A[i, j] = d

    # p = 2 corresponds to gaussian kernel. p = 1 corresponds to Laplacian kernel.
    if p == 2:  # Store squared euclidean for L2 distance otherwise if p = 1 just store absolute dist.
        A = A ** 2

            
    return A

import scipy as sp

# Compute weight matrix (i.e., the Graph Laplacian) $L$ for each set of question and its candidate answers
def get_weight_matrix(input_file, n_neighbors=5, sigma=1.0, eps=0.0001, p=2):
    """Compute weight matrix for question and answer sentences 
    """
    with open(input_file) as infile:
        num_questions = 0
        while infile:
            group = get_QA_group(infile)

            # Check for EOF
            if group is None:
                break
                
            # Extract question vector
            question = group["question"]
            qvec = extract_vector(question[0], words_to_exclude, w2idx, vectors)

            scores = []
            answer_vectors = []
            for (label, sentence) in group["answers"]:
                # Compute similarity with question vector
                vec = extract_vector(sentence, words_to_exclude, w2idx, vectors) # TODO: Pass these in as args
                answer_vectors.append(vec)
                cosine_distance = sp.spatial.distance.cosine(qvec, vec)
                scores.append((label, cosine_distance))

            # Compute pairwise distances between the answer vectors for K nearest neighbor
            k = min(n_neighbors, len(answer_vectors) - 1) # Minus 1 because have to exclude itself
            # Not enough to do rank propagation. Just keep original scores.
            if k < 0:
                yield None, None
            elif k == 0:
                yield 1, None
            else:                
                answer_vectors = np.vstack(answer_vectors)
                W = compute_pairwise_distance_matrix(answer_vectors, k, p)
                W = np.maximum(W, W.T)  # Ensure W symmetric.
                W[W > 0] = np.exp(- W[W > 0] / (2 * sigma**2))  # Apply gaussian kernel
                D = np.diag(np.sum(W, axis=1))  # Row sum of W
                L = D - W
#                 L = L + eps * np.eye(len(answer_vectors))  # Improve the condition of the graph laplacian                
                Dinvsqrt = np.sqrt(np.linalg.pinv(D))                
                # Need to ensure that Dinvsqrt does not have NAN due to division by 0
                assert(not np.any(np.isnan(Dinvsqrt)))                
                L = Dinvsqrt.dot(L).dot(Dinvsqrt)  # Normalized graph laplacian
                
#                 assert(is_pos_def(Dinvsqrt))
#                 assert(is_pos_def(L))
                
                yield L.shape[0], L
            
            num_questions += 1

def load_similarity_features(filepath):
    """
    Load question answer similarity values from file (probably should compute it here).

    **NOTE**: Similarity is the wrong term to use here. The values in the file are actually cosine **distances**.
    To convert it to similarity, we need to subtract it from 1.
    """
    
    features = []
    labels = []
    map_label = {"positive": 1, "negative": 0}
    with open(filepath) as infile:
        for line in infile:
            label, score = line.strip().split(',')
            score = float(score)
            label = map_label[label]
            features.append(score)
            labels.append(label)
            
    return np.asarray(features).reshape(-1, 1), np.asarray(labels)

In [13]:
train_file = "../data/answerSelectionExperiments/data/train-less-than-40.xml"
dev_file = "../data/answerSelectionExperiments/data/dev-less-than-40.xml"
test_file = "../data/answerSelectionExperiments/data/test-less-than-40.xml"
dev_qn_group_indicators = get_QA_group_indicators(dev_file)
qn_group_indicators = get_QA_group_indicators(test_file)

In [14]:
#import training data and test data. 
# These are the Jacana features for each qn/ans pair in the TREC dataset (XML files).
train_datapath = "../myclassify/qa.train.arff"
test_datapath = "../myclassify/qa.test.arff"
dev_datapath = "../myclassify/qa.dev.arff"

X_train, y_train = importData(train_datapath)
X_test, y_test = importData(test_datapath)
X_dev, y_dev = importData(dev_datapath)

In [15]:
X_sim_train, y_sim_train = load_similarity_features("../data/features/glove_embedding_sentence_similarities_train_300.txt")
X_sim_dev, y_sim_dev = load_similarity_features("../data/features/glove_embedding_sentence_similarities_dev_300.txt")
X_sim_test, y_sim_test = load_similarity_features("../data/features/glove_embedding_sentence_similarities_test_300.txt")

X_combined_train = np.hstack((X_train, X_sim_train))
X_combined_dev = np.hstack((X_dev, X_sim_dev))
X_combined_test = np.hstack((X_test, X_sim_test))

# Scale combined data
scaler = RobustScaler()
scaler.fit(X_combined_train)
X_comb_scaled_train = scaler.transform(X_combined_train)
X_comb_scaled_dev = scaler.transform(X_combined_dev)
X_comb_scaled_test = scaler.transform(X_combined_test)

# Only normalize the similarity scores
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler.fit(X_sim_train)
X_sim_train = scaler.transform(X_sim_train)
X_sim_dev = scaler.transform(X_sim_dev)
X_sim_test = scaler.transform(X_sim_test)
X_comb_scaledsim_train = np.hstack((X_train, X_sim_train))
X_comb_scaledsim_dev = np.hstack((X_dev, X_sim_dev))
X_comb_scaledsim_test = np.hstack((X_test, X_sim_test))

---
## Propagate rank score


In [16]:
from cvxpy import Variable, Minimize, norm, quad_form, Problem

def is_pos_def(x):
    """Check if a matrix is positive definite. For debugging purposes."""
    return np.all(np.linalg.eigvals(x) > 0)

# Version 2 that uses the ans_type_match and question answer similarity weights
def propagate_scores(r, L, ans_type_match, ans_sim_weights, alpha=1.0, gamma=1.0, loss_type=1):
    """Solve convex optimization problem to get new scores"""
        
    n = r.size
    y = Variable(n)    
    assert(len(ans_type_match) == n and len(ans_sim_weights) == n)
    
    # If no type match we just ignore the type term
    if not np.any(ans_type_match):
        objective = Minimize( norm(r - y, loss_type) + alpha * quad_form(y, L) )
    else:
        type_term = sum( ans_sim_weights[i] * cvxpy.abs(1 - y[i]) 
                        for i, match in enumerate(ans_type_match) if match == 1 )
        objective = Minimize( norm(r - y, loss_type) + alpha * quad_form(y, L) + gamma * type_term)
            
    constraints = [0 <= y, y <= 1]
    prob = Problem(objective, constraints)    

    # The optimal objective is returned by prob.solve().
    result = prob.solve(verbose=False)      
    assert(prob.status == "optimal")
    
    return y.value.flatten().tolist()[0]

def get_qn_answer_match_indicators(filepath):
    with open(filepath) as infile:
        return np.asarray([int(x.strip()) for x in infile])
    
def get_qn_answer_sim_weights(filepath):
    with open(filepath) as infile:
        return np.asarray([float(x.split(',')[1]) for x in infile])

def rank_propagation(data_filepath, qn_match_filepath, qn_simweights_filepath, r, 
                     alpha=1.0, sigma=1.0, n_neighbors=5, gamma=1.0, 
                     loss_type=1, 
                     pair_similarity_type=2):
    total_count = 0

    # Get qn group indicator
    qn_group_indicators = get_QA_group_indicators(data_filepath)
    
    # Get qn answer type match
    qn_ans_type_match = get_qn_answer_match_indicators(qn_match_filepath)
    
    # Get qn answer similarity weights
    # The weights are actually distances so we subtract them from 1 to convert distance to similarity
    qn_ans_sim_weights = 1 - get_qn_answer_sim_weights(qn_simweights_filepath)
    
    qn_number = 0  # Current question number (NOTE: This is not ID in XML)

    scores = []  # To store the final refined scores
    # L is actually the graph Laplacian matrix
    for (count, L) in get_weight_matrix(data_filepath, 
                                        n_neighbors, sigma, 
                                        p=pair_similarity_type):
        # Skip question without candidate answers
        if count is None:
            continue

        # Not enough points to propagate. Just use original value.
        MIN_NUM_CANDIDATES = 1
        if count <= MIN_NUM_CANDIDATES:
            assert(r[qn_group_indicators == qn_number].size == count)
            scores += r[qn_group_indicators == qn_number].tolist()
        else:
            # Get indicator vector for which answer has matching type
            ans_type_match = qn_ans_type_match[qn_group_indicators == qn_number]
            
            # Get question / answer similarity weights
            ans_sim_weights = qn_ans_sim_weights[qn_group_indicators == qn_number]                        
            
            # Propagate and append new scores
            assert(r[qn_group_indicators == qn_number].size == L.shape[0])
            new_scores = propagate_scores(r[qn_group_indicators == qn_number],
                                          L, 
                                          ans_type_match, ans_sim_weights,
                                          alpha, gamma, loss_type)                
            scores += new_scores

        qn_number += 1
        total_count += count

    return np.asarray(scores)

## Train basic classifier to give input ranks for Rank Propagation

In [18]:
from sklearn.metrics import f1_score, accuracy_score

In [19]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB

# nb = GaussianNB()
paras = [0.01, 0.1, 1, 2, 10, 50, 100]
for para in paras:
    nb = BernoulliNB(alpha=para)
    nb.fit(X_train, y_train)

    #dev set
    y_dev_pred = nb.predict(X_dev)
    y_dev_prob = nb.predict_proba(X_dev)
    dev_path = "../myclassify/test_res/NB-dev/NB-" + str(para) + ".txt"
    predict_for_test(y_dev, y_dev_pred, y_dev_prob, dev_path)

    #test set
    y_pred = nb.predict(X_test)
    y_prob = nb.predict_proba(X_test)
    test_path = "../myclassify/test_res/NB-test/NB-" + str(para) + ".txt"
    predict_for_test(y_test, y_pred, y_prob, test_path)

    #test result
    print("alpha: " + str(para))
    print("accuracy: {}".format(accuracy_score(y_test, y_pred)))
    print("f1: {}".format(f1_score(y_test, y_pred)))
#     print(confusion_matrix(y_test, y_pred))
    
#best MAP/MRR for dev when para = 2

alpha: 0.01
accuracy: 0.7323665128543178
f1: 0.3164983164983165
alpha: 0.1
accuracy: 0.7336849044166117
f1: 0.31756756756756754
alpha: 1
accuracy: 0.7350032959789057
f1: 0.3116438356164384
alpha: 2
accuracy: 0.7356624917600527
f1: 0.3050259965337955
alpha: 10
accuracy: 0.7554383651944627
f1: 0.2851637764932563
alpha: 50
accuracy: 0.8002636783124588
f1: 0.15598885793871867
alpha: 100
accuracy: 0.8127883981542519
f1: 0.08387096774193548


In [20]:
from sklearn.ensemble import GradientBoostingClassifier

# Probably don't try this.
# GBM
paras1 = [50, 100, 400, 800]
paras2 = [2, 3, 6, 12, 24]
for para1 in paras1:
    for para2 in paras2:    
        gbm = GradientBoostingClassifier(n_estimators=para1, max_depth=para2, random_state=47156)
        gbm.fit(X_train, y_train)
        
        #dev set
        y_dev_pred = gbm.predict(X_dev)
        y_dev_prob = gbm.predict_proba(X_dev)
        dev_path = "../myclassify/test_res/GB-dev/GB-" + str(para1) + "-" + str(para2) + ".txt"
        predict_for_test(y_dev, y_dev_pred, y_dev_prob, dev_path)

        #test set
        y_pred = gbm.predict(X_test)
        y_prob = gbm.predict_proba(X_test)
        test_path = "../myclassify/test_res/GB-test/GB-" + str(para1) + "-" + str(para2) + ".txt"
        predict_for_test(y_test, y_pred, y_prob, test_path)
        
        #test result
        print("n_estimators: " + str(para1) + "; max_depth: " + str(para2))
        print("accuracy: {}".format(accuracy_score(y_test, y_pred)))
        print("f1: {}".format(f1_score(y_test, y_pred)))
#         print(confusion_matrix(y_test, y_pred))

#best MAP/MRR for dev when para1 = 100, para2 = 2

n_estimators: 50; max_depth: 2
accuracy: 0.8279499011206328
f1: 0.25641025641025644
n_estimators: 50; max_depth: 3
accuracy: 0.8272907053394858
f1: 0.3105263157894737
n_estimators: 50; max_depth: 6
accuracy: 0.8193803559657218
f1: 0.30808080808080807
n_estimators: 50; max_depth: 12
accuracy: 0.8174027686222808
f1: 0.2987341772151899
n_estimators: 50; max_depth: 24
accuracy: 0.7851021753460777
f1: 0.32083333333333336
n_estimators: 100; max_depth: 2
accuracy: 0.8312458800263678
f1: 0.31182795698924726
n_estimators: 100; max_depth: 3
accuracy: 0.8358602504943968
f1: 0.38518518518518524
n_estimators: 100; max_depth: 6
accuracy: 0.8239947264337508
f1: 0.327455919395466
n_estimators: 100; max_depth: 12
accuracy: 0.8259723137771918
f1: 0.3333333333333333
n_estimators: 100; max_depth: 24
accuracy: 0.7864205669083718
f1: 0.31932773109243695
n_estimators: 400; max_depth: 2
accuracy: 0.8305866842452209
f1: 0.3591022443890274
n_estimators: 400; max_depth: 3
accuracy: 0.8305866842452209
f1: 0.38369

In [25]:
import itertools
# RF
estimators = [100, 150, 200, 300, 400]
criterions = ["gini", "entropy"]
depths = [8, 11, 16, 20]
splits = [2, 4, 6, 8]
weights = [None, "balanced", "balanced_subsample"]
para_list = [estimators, criterions, depths, splits, weights]
paras = list(itertools.product(*para_list))

for para in paras:
    rf = RandomForestClassifier(n_estimators=para[0], 
                                    criterion=para[1], 
                                    max_depth=para[2],
                                    min_samples_split = para[3],
                                    class_weight=para[4],
                                    random_state=73514)
    rf.fit(X_train, y_train)
    para_string = "-".join([str(p) for p in para])
    #dev set
    y_dev_pred = rf.predict(X_dev)
    y_dev_prob = rf.predict_proba(X_dev)
    dev_path = "../myclassify/test_res/RF-dev/RF-" + para_string + ".txt"
    predict_for_test(y_dev, y_dev_pred, y_dev_prob, dev_path)

    #test set
    y_pred = rf.predict(X_test)
    y_prob = rf.predict_proba(X_test)
    test_path = "../myclassify/test_res/RF-test/RF-" + para_string + ".txt"
    predict_for_test(y_test, y_pred, y_prob, test_path)

    #test result
    print(para_string)
    print("accuracy: {}".format(accuracy_score(y_test, y_pred)))
    print("f1: {}".format(f1_score(y_test, y_pred)))
#     print(confusion_matrix(y_test, y_pred))
    # predict_for_test(y_test, y_pred, rf.predict_proba(X_test), "rf.txt")
    
#best MAP/MRR for dev when 150-entropy-16-8-balanced

100-gini-8-2-None
accuracy: 0.8160843770599868
f1: 0.054237288135593226
100-gini-8-2-balanced
accuracy: 0.8035596572181938
f1: 0.4931972789115646
100-gini-8-2-balanced_subsample
accuracy: 0.7943309162821358
f1: 0.47474747474747475
100-gini-8-4-None
accuracy: 0.8174027686222808
f1: 0.07357859531772575
100-gini-8-4-balanced
accuracy: 0.7903757415952538
f1: 0.4803921568627451
100-gini-8-4-balanced_subsample
accuracy: 0.7864205669083718
f1: 0.4774193548387097
100-gini-8-6-None
accuracy: 0.8187211601845749
f1: 0.06143344709897611
100-gini-8-6-balanced
accuracy: 0.7903757415952538
f1: 0.4752475247524753
100-gini-8-6-balanced_subsample
accuracy: 0.7916941331575478
f1: 0.471571906354515
100-gini-8-8-None
accuracy: 0.8167435728411339
f1: 0.041379310344827586
100-gini-8-8-balanced
accuracy: 0.8002636783124588
f1: 0.48903878583473864
100-gini-8-8-balanced_subsample
accuracy: 0.7930125247198417
f1: 0.47840531561461797
100-gini-11-2-None
accuracy: 0.8167435728411339
f1: 0.06711409395973156
100-gini

150-gini-8-8-None
accuracy: 0.8180619644034278
f1: 0.05479452054794521
150-gini-8-8-balanced
accuracy: 0.8009228740936059
f1: 0.4932885906040268
150-gini-8-8-balanced_subsample
accuracy: 0.8009228740936059
f1: 0.49498327759197325
150-gini-11-2-None
accuracy: 0.8174027686222808
f1: 0.07357859531772575
150-gini-11-2-balanced
accuracy: 0.8266315095583389
f1: 0.3506172839506173
150-gini-11-2-balanced_subsample
accuracy: 0.8299274884640738
f1: 0.35820895522388063
150-gini-11-4-None
accuracy: 0.8200395517468688
f1: 0.08695652173913043
150-gini-11-4-balanced
accuracy: 0.8338826631509558
f1: 0.3883495145631068
150-gini-11-4-balanced_subsample
accuracy: 0.8319050758075148
f1: 0.38554216867469887
150-gini-11-6-None
accuracy: 0.8174027686222808
f1: 0.07357859531772575
150-gini-11-6-balanced
accuracy: 0.8299274884640738
f1: 0.3827751196172249
150-gini-11-6-balanced_subsample
accuracy: 0.8345418589321029
f1: 0.42032332563510394
150-gini-11-8-None
accuracy: 0.8174027686222808
f1: 0.06101694915254237

200-gini-11-6-None
accuracy: 0.8180619644034278
f1: 0.0738255033557047
200-gini-11-6-balanced
accuracy: 0.8292682926829268
f1: 0.3788968824940048
200-gini-11-6-balanced_subsample
accuracy: 0.8325642715886619
f1: 0.40375586854460094
200-gini-11-8-None
accuracy: 0.8180619644034278
f1: 0.0738255033557047
200-gini-11-8-balanced
accuracy: 0.8253131179960448
f1: 0.4097995545657016
200-gini-11-8-balanced_subsample
accuracy: 0.8266315095583389
f1: 0.40362811791383224
200-gini-16-2-None
accuracy: 0.8206987475280159
f1: 0.09333333333333332
200-gini-16-2-balanced
accuracy: 0.8272907053394858
f1: 0.19135802469135801
200-gini-16-2-balanced_subsample
accuracy: 0.8286090969017799
f1: 0.20245398773006137
200-gini-16-4-None
accuracy: 0.8180619644034278
f1: 0.09803921568627451
200-gini-16-4-balanced
accuracy: 0.8292682926829268
f1: 0.2359882005899705
200-gini-16-4-balanced_subsample
accuracy: 0.8305866842452209
f1: 0.23738872403560832
200-gini-16-6-None
accuracy: 0.8193803559657218
f1: 0.092715231788079

300-gini-16-4-None
accuracy: 0.8200395517468688
f1: 0.099009900990099
300-gini-16-4-balanced
accuracy: 0.8272907053394858
f1: 0.22485207100591711
300-gini-16-4-balanced_subsample
accuracy: 0.8292682926829268
f1: 0.2314540059347181
300-gini-16-6-None
accuracy: 0.8206987475280159
f1: 0.09933774834437085
300-gini-16-6-balanced
accuracy: 0.8325642715886619
f1: 0.26589595375722547
300-gini-16-6-balanced_subsample
accuracy: 0.8299274884640738
f1: 0.2543352601156069
300-gini-16-8-None
accuracy: 0.8187211601845749
f1: 0.08637873754152824
300-gini-16-8-balanced
accuracy: 0.8319050758075148
f1: 0.3013698630136986
300-gini-16-8-balanced_subsample
accuracy: 0.8332234673698088
f1: 0.29916897506925205
300-gini-20-2-None
accuracy: 0.8193803559657218
f1: 0.09271523178807946
300-gini-20-2-balanced
accuracy: 0.8233355306526038
f1: 0.12987012987012989
300-gini-20-2-balanced_subsample
accuracy: 0.8206987475280159
f1: 0.1111111111111111
300-gini-20-4-None
accuracy: 0.8167435728411339
f1: 0.0794701986754966

400-gini-20-2-None
accuracy: 0.8200395517468688
f1: 0.09302325581395347
400-gini-20-2-balanced
accuracy: 0.8220171390903098
f1: 0.11764705882352941
400-gini-20-2-balanced_subsample
accuracy: 0.8193803559657218
f1: 0.10457516339869281
400-gini-20-4-None
accuracy: 0.8180619644034278
f1: 0.08
400-gini-20-4-balanced
accuracy: 0.8286090969017799
f1: 0.18750000000000003
400-gini-20-4-balanced_subsample
accuracy: 0.8299274884640738
f1: 0.1987577639751553
400-gini-20-6-None
accuracy: 0.8193803559657218
f1: 0.09868421052631579
400-gini-20-6-balanced
accuracy: 0.8292682926829268
f1: 0.2127659574468085
400-gini-20-6-balanced_subsample
accuracy: 0.8279499011206328
f1: 0.21148036253776434
400-gini-20-8-None
accuracy: 0.8180619644034278
f1: 0.08
400-gini-20-8-balanced
accuracy: 0.8299274884640738
f1: 0.2543352601156069
400-gini-20-8-balanced_subsample
accuracy: 0.8332234673698088
f1: 0.27507163323782235
400-entropy-8-2-None
accuracy: 0.8160843770599868
f1: 0.054237288135593226
400-entropy-8-2-balanc

In [31]:
# LR

from sklearn.linear_model import LogisticRegression

# lr = LogisticRegression(C=3, class_weight="balanced")
# lr = LogisticRegression(C=0.01)
penalties = ["l1", "l2"]
cs = [0.01, 0.1, 1, 10, 100, 1000]

for para1 in penalties:
    for para2 in cs:
        lr = LogisticRegression(penalty = para1, C=para2, max_iter=1e8)
        lr.fit(X_train, y_train)
    
        #dev set
        y_dev_pred = lr.predict(X_dev)
        y_dev_prob = lr.predict_proba(X_dev)
        dev_path = "../myclassify/test_res/LG-dev/LG-" + str(para1) + "-" + str(para2) + ".txt"
        predict_for_test(y_dev, y_dev_pred, y_dev_prob, dev_path)

        #test set
        y_pred = lr.predict(X_test)
        y_prob = lr.predict_proba(X_test)
        test_path = "../myclassify/test_res/LG-test/LG-" + str(para1) + "-" + str(para2) + ".txt"
        predict_for_test(y_test, y_pred, y_prob, test_path)
        
        #test result
        print("penalty: " + str(para1) + "; C: " + str(para2))
        print("accuracy: {}".format(accuracy_score(y_test, y_pred)))
        print("f1: {}".format(f1_score(y_test, y_pred)))
#         print(confusion_matrix(y_test, y_pred))

#best MAP/MRR for dev when l1-10

penalty: l1; C: 0.01
accuracy: 0.8299274884640738
f1: 0.20858895705521474
penalty: l1; C: 0.1
accuracy: 0.8325642715886619
f1: 0.27011494252873564
penalty: l1; C: 1
accuracy: 0.8286090969017799
f1: 0.24418604651162787
penalty: l1; C: 10
accuracy: 0.8286090969017799
f1: 0.24418604651162787
penalty: l1; C: 100
accuracy: 0.8292682926829268
f1: 0.24489795918367346
penalty: l1; C: 1000
accuracy: 0.8292682926829268
f1: 0.24489795918367346
penalty: l2; C: 0.01
accuracy: 0.8378378378378378
f1: 0.28488372093023256
penalty: l2; C: 0.1
accuracy: 0.8305866842452209
f1: 0.2636103151862464
penalty: l2; C: 1
accuracy: 0.8279499011206328
f1: 0.2434782608695652
penalty: l2; C: 10
accuracy: 0.8279499011206328
f1: 0.239067055393586
penalty: l2; C: 100
accuracy: 0.8266315095583389
f1: 0.23768115942028983
penalty: l2; C: 1000
accuracy: 0.8279499011206328
f1: 0.239067055393586


## Train with SIM features

In [32]:
X_sim_train, y_sim_train = load_similarity_features("../data/features/glove_embedding_sentence_similarities_train_300.txt")
X_sim_dev, y_sim_dev = load_similarity_features("../data/features/glove_embedding_sentence_similarities_dev_300.txt")
X_sim_test, y_sim_test = load_similarity_features("../data/features/glove_embedding_sentence_similarities_test_300.txt")

# Only normalize the similarity scores
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler.fit(X_sim_train)
X_sim_train = scaler.transform(X_sim_train)
X_sim_dev = scaler.transform(X_sim_dev)
X_sim_test = scaler.transform(X_sim_test)
X_comb_scaledsim_train = np.hstack((X_train, X_sim_train))
X_comb_scaledsim_dev = np.hstack((X_dev, X_sim_dev))
X_comb_scaledsim_test = np.hstack((X_test, X_sim_test))

In [33]:
penalties = ["l1", "l2"]
cs = [0.01, 0.1, 1, 10, 100, 1000]

for para1 in penalties:
    for para2 in cs:
        lr = LogisticRegression(penalty = para1, C=para2, max_iter=1e8)
        lr.fit(X_comb_scaledsim_train, y_train)
    
        #dev set
        y_dev_pred = lr.predict(X_comb_scaledsim_dev)
        y_dev_prob = lr.predict_proba(X_comb_scaledsim_dev)
        dev_path = "../myclassify/test_res/LGSIM-dev/LGSIM-" + str(para1) + "-" + str(para2) + ".txt"
        predict_for_test(y_dev, y_dev_pred, y_dev_prob, dev_path)

        #test set
        y_pred = lr.predict(X_comb_scaledsim_test)
        y_prob = lr.predict_proba(X_comb_scaledsim_test)
        test_path = "../myclassify/test_res/LGSIM-test/LGSIM-" + str(para1) + "-" + str(para2) + ".txt"
        predict_for_test(y_test, y_pred, y_prob, test_path)
        
        #test result
        print("penalty: " + str(para1) + "; C: " + str(para2))
        print("accuracy: {}".format(accuracy_score(y_test, y_pred)))
        print("f1: {}".format(f1_score(y_test, y_pred)))
#         print(confusion_matrix(y_test, y_pred))

#best for dev: l2-0.1

penalty: l1; C: 0.01
accuracy: 0.8325642715886619
f1: 0.23493975903614459
penalty: l1; C: 0.1
accuracy: 0.8411338167435728
f1: 0.3573333333333333
penalty: l1; C: 1
accuracy: 0.8404746209624259
f1: 0.35638297872340424
penalty: l1; C: 10
accuracy: 0.8417930125247198
f1: 0.3782383419689119
penalty: l1; C: 100
accuracy: 0.8411338167435728
f1: 0.370757180156658
penalty: l1; C: 1000
accuracy: 0.8411338167435728
f1: 0.370757180156658
penalty: l2; C: 0.01
accuracy: 0.8437705998681608
f1: 0.33983286908078
penalty: l2; C: 0.1
accuracy: 0.8411338167435728
f1: 0.370757180156658
penalty: l2; C: 1
accuracy: 0.8398154251812788
f1: 0.3622047244094488
penalty: l2; C: 10
accuracy: 0.8404746209624259
f1: 0.36649214659685864
penalty: l2; C: 100
accuracy: 0.8404746209624259
f1: 0.3697916666666667
penalty: l2; C: 1000
accuracy: 0.8398154251812788
f1: 0.36553524804177545


In [35]:
import itertools
# RF
estimators = [100, 150, 200, 300, 400]
criterions = ["gini", "entropy"]
depths = [8, 11, 16, 20]
splits = [2, 4, 6, 8]
weights = [None, "balanced", "balanced_subsample"]
para_list = [estimators, criterions, depths, splits, weights]
paras = list(itertools.product(*para_list))

for para in paras:
    rf = RandomForestClassifier(n_estimators=para[0], 
                                    criterion=para[1], 
                                    max_depth=para[2],
                                    min_samples_split = para[3],
                                    class_weight=para[4],
                                    random_state=73514)
    rf.fit(X_comb_scaledsim_train, y_train)
    para_string = "-".join([str(p) for p in para])
    #dev set
    y_dev_pred = rf.predict(X_comb_scaledsim_dev)
    y_dev_prob = rf.predict_proba(X_comb_scaledsim_dev)
    dev_path = "../myclassify/test_res/RFSIM-dev/RFSIM-" + para_string + ".txt"
    predict_for_test(y_dev, y_dev_pred, y_dev_prob, dev_path)

    #test set
    y_pred = rf.predict(X_comb_scaledsim_test)
    y_prob = rf.predict_proba(X_comb_scaledsim_test)
    test_path = "../myclassify/test_res/RFSIM-test/RFSIM-" + para_string + ".txt"
    predict_for_test(y_test, y_pred, y_prob, test_path)

    #test result
    print(para_string)
    print("accuracy: {}".format(accuracy_score(y_test, y_pred)))
    print("f1: {}".format(f1_score(y_test, y_pred)))
#     print(confusion_matrix(y_test, y_pred))
    # predict_for_test(y_test, y_pred, rf.predict_proba(X_test), "rf.txt")
    
#best MAP/MRR for dev when 300-entropy-16-8-balanced_subsample

100-gini-8-2-None
accuracy: 0.8220171390903098
f1: 0.10596026490066225
100-gini-8-2-balanced
accuracy: 0.8141067897165458
f1: 0.49462365591397855
100-gini-8-2-balanced_subsample
accuracy: 0.8174027686222808
f1: 0.5097345132743364
100-gini-8-4-None
accuracy: 0.8220171390903098
f1: 0.10596026490066225
100-gini-8-4-balanced
accuracy: 0.8127883981542519
f1: 0.5069444444444444
100-gini-8-4-balanced_subsample
accuracy: 0.8187211601845749
f1: 0.5166959578207382
100-gini-8-6-None
accuracy: 0.8213579433091628
f1: 0.11726384364820847
100-gini-8-6-balanced
accuracy: 0.8200395517468688
f1: 0.521891418563923
100-gini-8-6-balanced_subsample
accuracy: 0.8266315095583389
f1: 0.5361552028218696
100-gini-8-8-None
accuracy: 0.8213579433091628
f1: 0.09966777408637874
100-gini-8-8-balanced
accuracy: 0.8206987475280159
f1: 0.5211267605633803
100-gini-8-8-balanced_subsample
accuracy: 0.8239947264337508
f1: 0.5257548845470693
100-gini-11-2-None
accuracy: 0.8246539222148979
f1: 0.17901234567901236
100-gini-11-

150-gini-8-8-None
accuracy: 0.8239947264337508
f1: 0.1188118811881188
150-gini-8-8-balanced
accuracy: 0.8174027686222808
f1: 0.5097345132743364
150-gini-8-8-balanced_subsample
accuracy: 0.8233355306526038
f1: 0.516245487364621
150-gini-11-2-None
accuracy: 0.8266315095583389
f1: 0.19076923076923077
150-gini-11-2-balanced
accuracy: 0.8266315095583389
f1: 0.3693045563549161
150-gini-11-2-balanced_subsample
accuracy: 0.8352010547132498
f1: 0.3932038834951457
150-gini-11-4-None
accuracy: 0.8259723137771918
f1: 0.1901840490797546
150-gini-11-4-balanced
accuracy: 0.8332234673698088
f1: 0.4074941451990632
150-gini-11-4-balanced_subsample
accuracy: 0.8332234673698088
f1: 0.4129930394431555
150-gini-11-6-None
accuracy: 0.8239947264337508
f1: 0.16822429906542058
150-gini-11-6-balanced
accuracy: 0.8319050758075148
f1: 0.42437923250564336
150-gini-11-6-balanced_subsample
accuracy: 0.8332234673698088
f1: 0.42630385487528344
150-gini-11-8-None
accuracy: 0.8259723137771918
f1: 0.1592356687898089
150-g

200-gini-11-6-None
accuracy: 0.8246539222148979
f1: 0.17901234567901236
200-gini-11-6-balanced
accuracy: 0.8365194462755439
f1: 0.44144144144144143
200-gini-11-6-balanced_subsample
accuracy: 0.8319050758075148
f1: 0.41913439635535304
200-gini-11-8-None
accuracy: 0.8272907053394858
f1: 0.18125000000000002
200-gini-11-8-balanced
accuracy: 0.8305866842452209
f1: 0.4351648351648352
200-gini-11-8-balanced_subsample
accuracy: 0.8358602504943968
f1: 0.44789356984478934
200-gini-16-2-None
accuracy: 0.8239947264337508
f1: 0.18348623853211007
200-gini-16-2-balanced
accuracy: 0.8319050758075148
f1: 0.247787610619469
200-gini-16-2-balanced_subsample
accuracy: 0.8325642715886619
f1: 0.2573099415204678
200-gini-16-4-None
accuracy: 0.8266315095583389
f1: 0.20060790273556234
200-gini-16-4-balanced
accuracy: 0.8332234673698088
f1: 0.29526462395543174
200-gini-16-4-balanced_subsample
accuracy: 0.8345418589321029
f1: 0.2929577464788733
200-gini-16-6-None
accuracy: 0.8272907053394858
f1: 0.196319018404907

300-gini-16-4-None
accuracy: 0.8253131179960448
f1: 0.19452887537993924
300-gini-16-4-balanced
accuracy: 0.8305866842452209
f1: 0.276056338028169
300-gini-16-4-balanced_subsample
accuracy: 0.8358602504943968
f1: 0.2985915492957747
300-gini-16-6-None
accuracy: 0.8266315095583389
f1: 0.19076923076923077
300-gini-16-6-balanced
accuracy: 0.8345418589321029
f1: 0.3197831978319783
300-gini-16-6-balanced_subsample
accuracy: 0.8332234673698088
f1: 0.3068493150684931
300-gini-16-8-None
accuracy: 0.8266315095583389
f1: 0.19571865443425077
300-gini-16-8-balanced
accuracy: 0.8371786420566908
f1: 0.36175710594315247
300-gini-16-8-balanced_subsample
accuracy: 0.8338826631509558
f1: 0.3333333333333333
300-gini-20-2-None
accuracy: 0.8266315095583389
f1: 0.2054380664652568
300-gini-20-2-balanced
accuracy: 0.8332234673698088
f1: 0.22629969418960244
300-gini-20-2-balanced_subsample
accuracy: 0.8319050758075148
f1: 0.22018348623853212
300-gini-20-4-None
accuracy: 0.8239947264337508
f1: 0.18348623853211007

400-gini-20-2-None
accuracy: 0.8266315095583389
f1: 0.20060790273556234
400-gini-20-2-balanced
accuracy: 0.8325642715886619
f1: 0.22560975609756095
400-gini-20-2-balanced_subsample
accuracy: 0.8325642715886619
f1: 0.22085889570552147
400-gini-20-4-None
accuracy: 0.8266315095583389
f1: 0.20060790273556234
400-gini-20-4-balanced
accuracy: 0.8312458800263678
f1: 0.25581395348837205
400-gini-20-4-balanced_subsample
accuracy: 0.8325642715886619
f1: 0.2529411764705882
400-gini-20-6-None
accuracy: 0.8266315095583389
f1: 0.19076923076923077
400-gini-20-6-balanced
accuracy: 0.8358602504943968
f1: 0.2985915492957747
400-gini-20-6-balanced_subsample
accuracy: 0.8305866842452209
f1: 0.2678062678062678
400-gini-20-8-None
accuracy: 0.8253131179960448
f1: 0.18461538461538463
400-gini-20-8-balanced
accuracy: 0.8338826631509558
f1: 0.31521739130434784
400-gini-20-8-balanced_subsample
accuracy: 0.8325642715886619
f1: 0.3135135135135135
400-entropy-8-2-None
accuracy: 0.8226763348714569
f1: 0.135048231511

In [36]:
from sklearn.ensemble import GradientBoostingClassifier

# Probably don't try this.
# GBM
paras1 = [50, 100, 400, 800]
paras2 = [2, 3, 6, 12, 24]
for para1 in paras1:
    for para2 in paras2:    
        gbm = GradientBoostingClassifier(n_estimators=para1, max_depth=para2, random_state=47156)
        gbm.fit(X_comb_scaledsim_train, y_train)
        
        #dev set
        y_dev_pred = gbm.predict(X_comb_scaledsim_dev)
        y_dev_prob = gbm.predict_proba(X_comb_scaledsim_dev)
        dev_path = "../myclassify/test_res/GBSIM-dev/GBSIM-" + str(para1) + "-" + str(para2) + ".txt"
        predict_for_test(y_dev, y_dev_pred, y_dev_prob, dev_path)

        #test set
        y_pred = gbm.predict(X_comb_scaledsim_test)
        y_prob = gbm.predict_proba(X_comb_scaledsim_test)
        test_path = "../myclassify/test_res/GBSIM-test/GBSIM-" + str(para1) + "-" + str(para2) + ".txt"
        predict_for_test(y_test, y_pred, y_prob, test_path)
        
        #test result
        print("n_estimators: " + str(para1) + "; max_depth: " + str(para2))
        print("accuracy: {}".format(accuracy_score(y_test, y_pred)))
        print("f1: {}".format(f1_score(y_test, y_pred)))
#         print(confusion_matrix(y_test, y_pred))

#best MAP/MRR for dev when para1 = 50, para2 = 2

n_estimators: 50; max_depth: 2
accuracy: 0.8391562294001318
f1: 0.37113402061855666
n_estimators: 50; max_depth: 3
accuracy: 0.8384970336189849
f1: 0.4067796610169491
n_estimators: 50; max_depth: 6
accuracy: 0.8035596572181938
f1: 0.3659574468085106
n_estimators: 50; max_depth: 12
accuracy: 0.8029004614370469
f1: 0.380952380952381
n_estimators: 50; max_depth: 24
accuracy: 0.7870797626895187
f1: 0.3962616822429907
n_estimators: 100; max_depth: 2
accuracy: 0.8391562294001318
f1: 0.3838383838383838
n_estimators: 100; max_depth: 3
accuracy: 0.8404746209624259
f1: 0.4182692307692307
n_estimators: 100; max_depth: 6
accuracy: 0.8101516150296638
f1: 0.3924050632911392
n_estimators: 100; max_depth: 12
accuracy: 0.8134475939353988
f1: 0.38876889848812096
n_estimators: 100; max_depth: 24
accuracy: 0.7923533289386948
f1: 0.411214953271028
n_estimators: 400; max_depth: 2
accuracy: 0.8371786420566908
f1: 0.3990267639902676
n_estimators: 400; max_depth: 3
accuracy: 0.8319050758075148
f1: 0.4110854503

In [37]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB

# nb = GaussianNB()
paras = [0.01, 0.1, 1, 2, 10, 50, 100]
for para in paras:
    nb = BernoulliNB(alpha=para)
    nb.fit(X_comb_scaledsim_train, y_train)

    #dev set
    y_dev_pred = nb.predict(X_comb_scaledsim_dev)
    y_dev_prob = nb.predict_proba(X_comb_scaledsim_dev)
    dev_path = "../myclassify/test_res/NBSIM-dev/NBSIM-" + str(para) + ".txt"
    predict_for_test(y_dev, y_dev_pred, y_dev_prob, dev_path)

    #test set
    y_pred = nb.predict(X_comb_scaledsim_test)
    y_prob = nb.predict_proba(X_comb_scaledsim_test)
    test_path = "../myclassify/test_res/NBSIM-test/NBSIM-" + str(para) + ".txt"
    predict_for_test(y_test, y_pred, y_prob, test_path)

    #test result
    print("alpha: " + str(para))
    print("accuracy: {}".format(accuracy_score(y_test, y_pred)))
    print("f1: {}".format(f1_score(y_test, y_pred)))
#     print(confusion_matrix(y_test, y_pred))
    
#best MAP/MRR for dev when para = 1

alpha: 0.01
accuracy: 0.7514831905075807
f1: 0.3747927031509121
alpha: 0.1
accuracy: 0.7514831905075807
f1: 0.3747927031509121
alpha: 1
accuracy: 0.7547791694133158
f1: 0.37583892617449666
alpha: 2
accuracy: 0.7508239947264338
f1: 0.36148648648648646
alpha: 10
accuracy: 0.7765326301911668
f1: 0.3468208092485549
alpha: 50
accuracy: 0.8068556361239289
f1: 0.16997167138810196
alpha: 100
accuracy: 0.8114700065919578
f1: 0.08333333333333333


## Test Rank Propagation
---

In [38]:
from sklearn.metrics import classification_report, confusion_matrix

In [39]:
#train classifiers using best parameters obtained above, Jacana features only

classifiers = {}

nb = BernoulliNB(alpha=2)
nb.fit(X_train, y_train)
classifiers["NB"] = nb

gbm = GradientBoostingClassifier(n_estimators=100, max_depth=2, random_state=47156)
gbm.fit(X_train, y_train)
classifiers["GB"] = gbm

rf = RandomForestClassifier(n_estimators=150, 
                                criterion="entropy", 
                                max_depth=16,
                                min_samples_split = 8,
                                class_weight="balanced",
                                random_state=73514)
rf.fit(X_train, y_train)
classifiers["RF"] = rf

lr = LogisticRegression(penalty = "l1", C=10, max_iter=1e8)
lr.fit(X_train, y_train)
classifiers["LG"] = lr

#train classifiers using best parameters obtained above, Jacana features with normalize the similarity scores

nbsim = BernoulliNB(alpha=1)
nbsim.fit(X_comb_scaledsim_train, y_train)
classifiers["NBSIM"] = nbsim

gbmsim = GradientBoostingClassifier(n_estimators=50, max_depth=2, random_state=47156)
gbmsim.fit(X_comb_scaledsim_train, y_train)
classifiers["GBSIM"] = gbmsim

rfsim = RandomForestClassifier(n_estimators=300, 
                                criterion="entropy", 
                                max_depth=16,
                                min_samples_split = 8,
                                class_weight="balanced_subsample",
                                random_state=73514)
rfsim.fit(X_comb_scaledsim_train, y_train)
classifiers["RFSIM"] = rfsim

lrsim = LogisticRegression(penalty = "l2", C=0.1, max_iter=1e8)
lrsim.fit(X_comb_scaledsim_train, y_train)

classifiers["LGSIM"] = lrsim


In [42]:
# For testing

# This file contains entries indicating whether answer has a entity matching type required by answer prediction
# Line corresponding to entry has 1 if there is a match, else 0
qn_match_filepath = "../data/QuestionType/test_answer_type_match.txt"

#dev
dev_qn_match_filepath = "../data/QuestionType/dev_answer_type_match.txt"

# This file contains the cosine *distance* between question and answer
qn_simweights_filepath = "../data/features/glove_embedding_sentence_similarities_test_300.txt"

#dev
dev_qn_simweights_filepath = "../data/features/glove_embedding_sentence_similarities_dev_300.txt"

import itertools    
for key in classifiers:
    # Set this to one of the trained classifiers above (after training)
    clf = classifiers[key]
    
    if "SIM" not in key:
        dev_raw_scores = clf.predict_proba(X_dev)[:, 1]
        dev_raw_full_scores = clf.predict_proba(X_dev)
        y_dev_pred = clf.predict(X_dev)

        raw_scores = clf.predict_proba(X_test)[:, 1]
        raw_full_scores = clf.predict_proba(X_test)
        y_pred = clf.predict(X_test)
        
    elif "SIM" in key:
        dev_raw_scores = clf.predict_proba(X_comb_scaledsim_dev)[:, 1]
        dev_raw_full_scores = clf.predict_proba(X_comb_scaledsim_dev)
        y_dev_pred = clf.predict(X_comb_scaledsim_dev)

        raw_scores = clf.predict_proba(X_comb_scaledsim_test)[:, 1]
        raw_full_scores = clf.predict_proba(X_comb_scaledsim_test)
        y_pred = clf.predict(X_comb_scaledsim_test)

# These are used with best model
# raw_scores = clf.predict_proba(X_comb_scaledsim_test)[:, 1]
# raw_full_scores = clf.predict_proba(X_comb_scaledsim_test)
# y_pred = clf.predict(X_comb_scaledsim_test)

# I think we have to keep the number of neighbors in the graph small because there are only a few
# positive examples. We don't want to link them to too many negative ones.
# This works well with 300 dim glove vector
# scores = rank_propagation(test_file, qn_match_filepath, qn_simweights_filepath, 
#                           raw_scores, alpha=1, sigma=1.0, n_neighbors=3, gamma=1.5)

# L2 loss for | r - y | gives 0.6377 and 0.755 
# scores = rank_propagation(test_file, qn_match_filepath, qn_simweights_filepath, 
#                           raw_scores, alpha=0.5, sigma=1, n_neighbors=3, gamma=1.5,
#                           loss_type=1,
#                           pair_similarity_type=2)

# MRR = 0.7474 vs 0.7399, MAP = 0.6462, 0.6272 for pair_similarity_type=2, sigma=1.0
# Generated with LR trained with C = 100, max_iter = 10000
# map            all 0.6448
# recip_rank     all 0.7537
# scores = rank_propagation(test_file, qn_match_filepath, qn_simweights_filepath, 
#                           raw_scores, alpha=0.5, sigma=2.0, n_neighbors=3, gamma=1.5,
#                           loss_type=1, 
#                           pair_similarity_type=1)

# This with a LR trained with C = 2 gives MRR = 0.7674 MAP = 0.6518
# For this to work, we have to use X_comb_scaledsim_test
# scores = rank_propagation(test_file, qn_match_filepath, qn_simweights_filepath, 
#                           raw_scores, alpha=2.0, sigma=1.0, n_neighbors=5, gamma=1.0,
#                           loss_type=1, 
#                           pair_similarity_type=2)

    alphas = [0.05, 0.1, 0.5, 1.0, 1.5, 2.0]
    sigmas = [0.5, 0.75, 1.0, 1.5, 2.0]
    neighbors = [3, 5]
    gammas = [0, 0.8, 1.0, 1.25, 1.5, 1.75, 2.0]
    loss_type = [1, 2]
    para_list = [alphas, sigmas, neighbors, gammas, loss_type]
    paras = list(itertools.product(*para_list))
    
    for para in paras:
        para_string = "-".join([str(p) for p in para])
        
        print("************************************************************")
        print("classifier: ", key)
        print("rank propagation parameters: ", para_string)
        print("************************************************************")

        print("==========================dev================================")
        
        dev_scores = rank_propagation(dev_file, dev_qn_match_filepath, dev_qn_simweights_filepath, 
                                  dev_raw_scores, alpha=para[0], sigma=para[1], n_neighbors=para[2], gamma=para[3],
                                  loss_type=para[4], 
                                  pair_similarity_type=2)

        y_dev_pred_adjusted = (dev_scores >= 0.5)

        print("Adjusted accuracy: {}".format(accuracy_score(y_dev, dev_scores >= 0.5)))
        print("Original accuracy: {}".format(accuracy_score(y_dev, y_dev_pred)))
        print("Adjusted f1: {}".format(f1_score(y_dev, dev_scores >= 0.5)))
        print("Original f1: {}".format(f1_score(y_dev, y_dev_pred)))
        print(np.sum(np.abs(dev_raw_scores - dev_scores)) / len(dev_scores))  # Average difference between actual
        print(np.max(np.abs(dev_raw_scores - dev_scores)))
        print(np.sum(y_test != (dev_scores >= 0.5)))
        print(np.sum(y_test != (dev_scores >= 0.5)) / float(len(dev_scores)))
        print("Number of disagreement: {}".format(np.sum(np.abs(y_dev_pred - y_dev_pred_adjusted))))
        print("Adjusted")
        print(classification_report(y_dev, y_dev_pred_adjusted, digits=4))
        print("Original")
        print(classification_report(y_dev, y_dev_pred, digits=4))
    
        # Convert dev to weka format
        P_dev = np.hstack(((1 - dev_scores).reshape(-1, 1), dev_scores.reshape(-1, 1)))
        dev_path = "../myclassify/test_res/RP" + key + "-dev/" + key + "-" + para_string + ".txt"
        
        ensure_dir_exists(dev_path)
        
        predict_for_test(y_dev, y_dev_pred_adjusted, P_dev, dev_path)
        #predict_for_test(y_test, y_dev_pred, dev_raw_full_scores, "nb_no_adjust.txt")

        print("==========================test================================")
        
        scores = rank_propagation(test_file, qn_match_filepath, qn_simweights_filepath, 
                                  raw_scores, alpha=para[0], sigma=para[1], n_neighbors=para[2], gamma=para[3],
                                  loss_type=para[4], 
                                  pair_similarity_type=2)

        y_pred_adjusted = (scores >= 0.5)
    

        print("Adjusted accuracy: {}".format(accuracy_score(y_test, scores >= 0.5)))
        print("Original accuracy: {}".format(accuracy_score(y_test, y_pred)))
        print("Adjusted f1: {}".format(f1_score(y_test, scores >= 0.5)))
        print("Original f1: {}".format(f1_score(y_test, y_pred)))
        print(np.sum(np.abs(raw_scores - scores)) / len(scores))  # Average difference between actual
        print(np.max(np.abs(raw_scores - scores)))
        print(np.sum(y_test != (scores >= 0.5)))
        print(np.sum(y_test != (scores >= 0.5)) / float(len(scores)))
        print("Number of disagreement: {}".format(np.sum(np.abs(y_pred - y_pred_adjusted))))
        print("Adjusted")
        print(classification_report(y_test, y_pred_adjusted, digits=4))
        print("Original")
        print(classification_report(y_test, y_pred, digits=4))
    
    
        # Convert test to weka format
        P = np.hstack(((1 - scores).reshape(-1, 1), scores.reshape(-1, 1)))
        test_path = "../myclassify/test_res/RP" + key + "-test/" + key + "-" + para_string + ".txt"
        predict_for_test(y_test, y_pred_adjusted, P, test_path)
        #predict_for_test(y_test, y_pred, raw_full_scores, "nb_no_adjust.txt")

************************************************************
classifier:  NB
rank propagation parameters:  0.05-0.5-3-0-1
************************************************************
==========================dev================================
Adjusted accuracy: 0.7543554006968641
Original accuracy: 0.7543554006968641
Adjusted f1: 0.2694300518134715
Original f1: 0.2694300518134715
1.3980779643e-12
1.44692234026e-10
1
0.000871080139373
Number of disagreement: 0
Adjusted
             precision    recall  f1-score   support

          0     0.8272    0.8790    0.8524       926
          1     0.3171    0.2342    0.2694       222

avg / total     0.7286    0.7544    0.7396      1148

Original
             precision    recall  f1-score   support

          0     0.8272    0.8790    0.8524       926
          1     0.3171    0.2342    0.2694       222

avg / total     0.7286    0.7544    0.7396      1148

==========================test================================


/Users/hongwei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:103: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
/Users/hongwei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:104: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.


Adjusted accuracy: 0.7356624917600527
Original accuracy: 0.7356624917600527
Adjusted f1: 0.3050259965337955
Original f1: 0.3050259965337955
1.29042389473e-12
3.51773153613e-10
401
0.26433750824
Number of disagreement: 0
Adjusted
             precision    recall  f1-score   support

          0     0.8399    0.8337    0.8368      1233
          1     0.3003    0.3099    0.3050       284

avg / total     0.7389    0.7357    0.7372      1517

Original
             precision    recall  f1-score   support

          0     0.8399    0.8337    0.8368      1233
          1     0.3003    0.3099    0.3050       284

avg / total     0.7389    0.7357    0.7372      1517

************************************************************
classifier:  NB
rank propagation parameters:  0.05-0.5-3-0-2
************************************************************
==========================dev================================
Adjusted accuracy: 0.7543554006968641
Original accuracy: 0.7543554006968641
Adjusted f1

Adjusted accuracy: 0.7053394858272907
Original accuracy: 0.7356624917600527
Adjusted f1: 0.38683127572016457
Original f1: 0.3050259965337955
0.10153586142
0.999999687311
447
0.294660514173
Number of disagreement: 160
Adjusted
             precision    recall  f1-score   support

          0     0.8666    0.7534    0.8061      1233
          1     0.3169    0.4965    0.3868       284

avg / total     0.7637    0.7053    0.7276      1517

Original
             precision    recall  f1-score   support

          0     0.8399    0.8337    0.8368      1233
          1     0.3003    0.3099    0.3050       284

avg / total     0.7389    0.7357    0.7372      1517

************************************************************
classifier:  NB
rank propagation parameters:  0.05-0.5-3-1.25-1
************************************************************
==========================dev================================
Adjusted accuracy: 0.7543554006968641
Original accuracy: 0.7543554006968641
Adjusted f1

Adjusted accuracy: 0.7336849044166117
Original accuracy: 0.7356624917600527
Adjusted f1: 0.40762463343108507
Original f1: 0.3050259965337955
0.0638874901915
0.999999681458
404
0.266315095583
Number of disagreement: 105
Adjusted
             precision    recall  f1-score   support

          0     0.8704    0.7899    0.8282      1233
          1     0.3492    0.4894    0.4076       284

avg / total     0.7729    0.7337    0.7495      1517

Original
             precision    recall  f1-score   support

          0     0.8399    0.8337    0.8368      1233
          1     0.3003    0.3099    0.3050       284

avg / total     0.7389    0.7357    0.7372      1517

************************************************************
classifier:  NB
rank propagation parameters:  0.05-0.5-3-1.75-2
************************************************************
==========================dev================================
Adjusted accuracy: 0.7029616724738676
Original accuracy: 0.7543554006968641
Adjusted 

...

Adjusted accuracy: 0.8312458800263678
Original accuracy: 0.8312458800263678
Adjusted f1: 0.31182795698924726
Original f1: 0.31182795698924726
2.83499065897e-05
0.043006796145
256
0.168754119974
Number of disagreement: 0
Adjusted
             precision    recall  f1-score   support

          0     0.8418    0.9757    0.9038      1233
          1     0.6591    0.2042    0.3118       284

avg / total     0.8076    0.8312    0.7930      1517

Original
             precision    recall  f1-score   support

          0     0.8418    0.9757    0.9038      1233
          1     0.6591    0.2042    0.3118       284

avg / total     0.8076    0.8312    0.7930      1517

************************************************************
classifier:  GB
rank propagation parameters:  0.5-1.5-3-1.0-2
************************************************************
==========================dev================================
Adjusted accuracy: 0.75
Original accuracy: 0.8101045296167247
Adjusted f1: 0.321513002

Adjusted accuracy: 0.8257839721254355
Original accuracy: 0.8101045296167247
Adjusted f1: 0.3464052287581699
Original f1: 0.1484375
0.0356601187997
0.974713448222
1
0.000871080139373
Number of disagreement: 50
Adjusted
             precision    recall  f1-score   support

          0     0.8412    0.9665    0.8995       926
          1     0.6310    0.2387    0.3464       222

avg / total     0.8005    0.8258    0.7925      1148

Original
             precision    recall  f1-score   support

          0     0.8178    0.9838    0.8931       926
          1     0.5588    0.0856    0.1484       222

avg / total     0.7677    0.8101    0.7491      1148

==========================test================================
Adjusted accuracy: 0.8292682926829268
Original accuracy: 0.8312458800263678
Adjusted f1: 0.4681724845995893
Original f1: 0.31182795698924726
0.0654113190454
0.969010005213
259
0.170731707317
Number of disagreement: 115
Adjusted
             precision    recall  f1-score   support

Adjusted accuracy: 0.8299274884640738
Original accuracy: 0.8312458800263678
Adjusted f1: 0.29120879120879123
Original f1: 0.31182795698924726
0.00445421171066
0.183279371077
258
0.170072511536
Number of disagreement: 8
Adjusted
             precision    recall  f1-score   support

          0     0.8392    0.9781    0.9034      1233
          1     0.6625    0.1866    0.2912       284

avg / total     0.8062    0.8299    0.7888      1517

Original
             precision    recall  f1-score   support

          0     0.8418    0.9757    0.9038      1233
          1     0.6591    0.2042    0.3118       284

avg / total     0.8076    0.8312    0.7930      1517

************************************************************
classifier:  GB
rank propagation parameters:  0.5-1.5-5-0.8-1
************************************************************
==========================dev================================
Adjusted accuracy: 0.8101045296167247
Original accuracy: 0.8101045296167247
Adjusted f1

Adjusted accuracy: 0.7360627177700348
Original accuracy: 0.8101045296167247
Adjusted f1: 0.33698030634573306
Original f1: 0.1484375
0.158590683209
0.977222413835
1
0.000871080139373
Number of disagreement: 203
Adjusted
             precision    recall  f1-score   support

          0     0.8412    0.8294    0.8352       926
          1     0.3277    0.3468    0.3370       222

avg / total     0.7419    0.7361    0.7389      1148

Original
             precision    recall  f1-score   support

          0     0.8178    0.9838    0.8931       926
          1     0.5588    0.0856    0.1484       222

avg / total     0.7677    0.8101    0.7491      1148

==========================test================================
Adjusted accuracy: 0.7554383651944627
Original accuracy: 0.8312458800263678
Adjusted f1: 0.404494382022472
Original f1: 0.31182795698924726
0.163668470956
0.985480375781
371
0.244561634806
Number of disagreement: 267
Adjusted
             precision    recall  f1-score   support


KeyboardInterrupt: 

LR with C = 1, max_iter = 10000
scores = rank_propagation(test_file, qn_match_filepath, qn_simweights_filepath, 
                          raw_scores, alpha=0.5, sigma=1, n_neighbors=3, gamma=1.5,
                          loss_type=1,
                          pair_similarity_type=2)

Adjusted
             precision    recall  f1-score   support

          0       0.85      0.98      0.91      1233
          1       0.74      0.26      0.38       284

avg / total       0.83      0.84      0.81      1517

Original
             precision    recall  f1-score   support

          0       0.83      0.98      0.90      1233
          1       0.69      0.15      0.24       284

avg / total       0.81      0.83      0.78      1517

MRR = 0.7474 vs 0.7399, MAP = 0.6462, 0.6272
LR trained with C = 100, max_iter = 10000

scores = rank_propagation(test_file, qn_match_filepath, qn_simweights_filepath, 
                          raw_scores, alpha=0.5, sigma=1, n_neighbors=3, gamma=1.5,
                          loss_type=1, 
                          pair_similarity_type=2)
                          
Adjusted
             precision    recall  f1-score   support

          0       0.85      0.98      0.91      1233
          1       0.73      0.24      0.36       284

avg / total       0.83      0.84      0.81      1517

Original
             precision    recall  f1-score   support

          0       0.83      0.98      0.90      1233
          1       0.68      0.14      0.24       284

avg / total       0.81      0.83      0.78      1517
                          

---

In [ ]:
# For training
train_raw_scores = clf.predict_proba(X_train)[:, 1]
scores = rank_propagation(train_file, train_raw_scores, alpha=2, sigma=2, n_neighbors=11)

print(accuracy_score(y_train, scores >= 0.5))
print(np.sum(np.abs(train_raw_scores - scores)) / len(scores))  # Average difference between actual

In [ ]:
accuracy_score(y_train, clf.predict(X_train))